In [1]:
import torchtext
import torch
from torch import nn
import spacy
import random
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.utils import get_tokenizer

In [8]:
a = torch.randn(2,3,4,5)
v, i = torch.max(a, dim=0)
v, i

(tensor([[[ 2.5351,  0.7386,  0.9047,  0.6718,  1.8967],
          [-0.6330,  0.6107,  1.7674,  1.1559, -0.6745],
          [-0.1930,  0.3196,  1.6610, -0.2573, -1.1351],
          [ 1.7515,  1.2135,  0.1067, -0.5647, -0.2714]],
 
         [[ 0.5914,  1.2536, -0.0083,  1.7683,  0.4249],
          [ 0.2459,  0.8544,  0.1151,  1.4737,  0.9920],
          [ 0.7062,  0.1403, -0.3241,  3.4067,  0.7415],
          [ 0.5486, -0.2074,  1.5844,  0.5252,  0.6802]],
 
         [[-0.8402,  0.2314,  0.7635, -0.9660, -0.0707],
          [ 2.0927,  0.9093,  1.1700,  2.1372,  0.8773],
          [-0.6008,  2.9211,  1.0879,  1.1569,  0.5145],
          [ 2.4891,  1.3333, -0.1935, -1.3532, -0.0552]]]),
 tensor([[[0, 0, 0, 0, 1],
          [0, 1, 0, 0, 1],
          [1, 0, 0, 0, 0],
          [1, 1, 0, 1, 0]],
 
         [[1, 0, 1, 0, 0],
          [0, 1, 1, 0, 1],
          [0, 1, 1, 1, 0],
          [0, 1, 0, 0, 1]],
 
         [[1, 0, 1, 1, 0],
          [0, 0, 0, 1, 0],
          [0, 1, 1, 1, 0],
    

In [2]:
import random
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torchtext.datasets import Multi30k
# from torchtext.datasets import IMDB
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator
# from torchtext.data import get_tokenizer

In [3]:
torchtext.__version__

'0.17.2+cpu'

In [4]:
train_iter = Multi30k(split=('train'))
next(iter(train_iter))

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.')

In [5]:
len(list(train_iter))

C:\Users\pouya\jupyter projects\ml_venv\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


29001

In [6]:
spacy_eng = spacy.load("en_core_web_sm")
spacy_ger = spacy.load("de_core_news_sm")
spacy_ger 

In [7]:
eng_tokenizer = get_tokenizer('basic_english')
spacy_eng = spacy.load("en_core_web_sm")
spacy_ger = spacy.load("de_core_news_sm")
ger_tokenizer = lambda x: [tok.text for tok in spacy_ger.tokenizer(x)]
# spacy_eng_tokenizer = lambda x: [tok.text for tok in spacy_eng.tokenizer(x)]

In [8]:
ger_tokenizer('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.')

['Zwei',
 'junge',
 'weiße',
 'Männer',
 'sind',
 'im',
 'Freien',
 'in',
 'der',
 'Nähe',
 'vieler',
 'Büsche',
 '.']

In [9]:
counter_train_eng = Counter()
for src, trg in train_iter:
    counter_train_eng.update(eng_tokenizer(trg))

counter_train_ger = Counter()
for src, trg in train_iter:
    counter_train_ger.update(ger_tokenizer(src))

In [10]:
eng = vocab(counter_train_eng, min_freq=2, specials=['<pad>', '<sos>', '<eos>', '<unk>'])
ger = vocab(counter_train_ger, min_freq=2, specials=['<pad>', '<sos>', '<eos>', '<unk>'])

print("The length of the new vocab is", len(eng))
new_stoi = eng.get_stoi()
print("The index of <sos> is", new_stoi['<sos>'])
new_itos = eng.get_itos()
print("The token at index 2 is", new_itos[5])

The length of the new vocab is 5921
The index of <sos> is 1
The token at index 2 is young


In [11]:
# eng.set_default_index(eng['<unk>'])
# eng['<unk>']

In [12]:
text_transform_eng = lambda x: [eng["<sos>"]] + [eng[token] for token in eng_tokenizer(x)] + [eng['<eos>']]
text_transform_ger = lambda x: [ger["<sos>"]] + [ger[token] for token in ger_tokenizer(x)] + [ger['<eos>']]
print(f"input: {next(iter(train_iter))[1]}")
print(f"output of text_transform: {text_transform_eng(next(iter(train_iter))[1])}")

input: Two young, White males are outside near many bushes.
output of text_transform: [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 2]


In [13]:
# def yield_tokens(examples):
#     for example in examples:
#         tokens = tokenizer(example)
# #         print('tokens:', tokens)
#         yield tokens

# text_list = ["hello world how are you doing",
#         "hi, world what is up and how are you doing world?"]
# list(yield_tokens(text_list))

In [14]:
# token_generator = yield_tokens(text_list)
# vocab = build_vocab_from_iterator(token_generator, min_freq=1, specials=['<pad>', '<sos>', '<eos>', '<unk>'],\
#                                  special_first=False)
# {k:v for k,v in sorted(vocab.get_stoi().items(), key=lambda x:x[1])}

In [15]:
# vocab.set_default_index(vocab['<unk>'])
# vocab.get_stoi()

In [16]:
# vocab['world']

In [17]:
# list(yield_tokens(text_list))[0]

In [18]:
# vocab.lookup_token(0)

In [19]:
# vocab.lookup_indices(list(yield_tokens(text_list))[0])

In [20]:
# vocab.state_dict()

In [21]:
# vocab.cuda()

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size, n_layers, p):
        super().__init__()
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, n_layers, dropout=p)
        
    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.rnn(embedding)
        return hidden, cell
        

In [23]:
encoder = Encoder(6, 400, 500, 3, 0.5)
encoder

Encoder(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(6, 500)
  (rnn): LSTM(500, 400, num_layers=3, dropout=0.5)
)

In [24]:
inp = torch.randint(0,2,(6,2))
hidden, cell = encoder(inp)
hidden.shape, cell.shape

(torch.Size([3, 2, 400]), torch.Size([3, 2, 400]))

In [25]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, p):
        super().__init__()
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden, cell):
        #we are sending one word rather than a sentence: N -> (1, N) for (sequence lenght, batch size)
        x = x.unsqueeze(0)
        x = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        #(seq length, N, output_size)
        predictions = self.fc(output)
        
        #(N, output_size)
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell
        

In [26]:
decoder = Decoder(10, 64, 128, 50, 1, 0.5)

In [27]:
x = torch.randint(0,5, (1, 50)).reshape(50)
h = torch.zeros(1, 50, 128)
c = torch.zeros(1, 50, 128)
decoder(x, h, c)[2].shape

torch.Size([1, 50, 128])

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)  ###debug
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        
        hidden, cell = self.encoder(source)
        
        x = target[0]
        for i in range(target_len):
            output , hidden, cell = self.decoder(x, hidden, cell)
            
            outputs[i] = output
            
            best_guess = output.argmax(dim=1) 
            
        x = target[i] if random.random() < teacher_force_ratio else best_guess
        return outputs

In [29]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
lr = 0.001
epochs = 100
batch_size = 64

In [30]:
input_size_encoder = 1000  ##debug len(vocab)
input_size_decoder = 1000  ##debug len(vocab)
output_size = 100
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5
writer = SummaryWriter()

In [31]:
list(train_iter)[0:3]

[('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
  'Two young, White males are outside near many bushes.'),
 ('Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
  'Several men in hard hats are operating a giant pulley system.'),
 ('Ein kleines Mädchen klettert in ein Spielhaus aus Holz.',
  'A little girl climbing into a wooden playhouse.')]

In [32]:
def collate_batch(batch):
    eng_list, ger_list = [], []
    for ger, eng in batch:
        processed_eng = torch.tensor(text_transform_eng(eng))
        eng_list.append(processed_eng)
        processed_ger = torch.tensor(text_transform_ger(ger))                            
        ger_list.append(processed_ger)        
    return pad_sequence(ger_list, padding_value=3.0), pad_sequence(eng_list, padding_value=3.0)

train_dataloader = DataLoader(list(train_iter), batch_size=batch_size, shuffle=True, collate_fn=collate_batch)

In [33]:
len(train_dataloader),type(train_dataloader)

(454, torch.utils.data.dataloader.DataLoader)

In [34]:
train_list = list(train_iter)
train_list[5]

('Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.',
 'A man in green holds a guitar while the other man observes his shirt.')

In [35]:
indices_ger = [(i, len(ger_tokenizer(s[0]))) for i, s in enumerate(train_list)]

def batch_sampler():
    indices_eng = [(i, len(ger_tokenizer(s[1]))) for i, s in enumerate(train_list)]
    random.shuffle(indices_eng)
    pooled_indices = []
    for i in range(0, len(indices_eng), batch_size*50):
        pooled_indices.extend(sorted(indices_eng[i: i+batch_size*50], key=lambda x: x[1]))

    pooled_indices = [x[0] for x in pooled_indices]

    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i+batch_size]

bucket_dataloader = DataLoader(train_list, batch_sampler=batch_sampler(),
                               collate_fn=collate_batch)

# next(iter(bucket_dataloader))


In [36]:
encoder = Encoder(input_size_encoder, hidden_size, encoder_embedding_size, num_layers, enc_dropout)
x = torch.randint(1,5, (10, 100))
encoder(x)[0].shape

torch.Size([2, 100, 1024])

In [37]:
dict([
    (2,'result1'),
    (1, 'result2')
])


{2: 'result1', 1: 'result2'}

In [38]:
import numpy as np
import random
lst = [[random.randint(4000,6000) for i in range(50)] for i in range(10)]
np.percentile(lst, 90,axis=1 ,keepdims=True)

array([[5738.7],
       [5721.9],
       [5811. ],
       [5797.9],
       [5802. ],
       [5782.1],
       [5647.5],
       [5912.7],
       [5757.8],
       [5685.1]])